# Model training

Let's start by joining the training features and putting in an adequate format for LightGBM.

In [26]:
import pathlib
import pandas as pd

features_dir = pathlib.Path('train_features/')

nrows = None
features = pd.concat(
    (
        pd.read_csv(f, index_col=0, nrows=nrows)
        for f in features_dir.glob('*.csv')
    ),
    axis='columns'
)

features.head()

/Users/mhalford/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,question_difficulty,avg_correct
row_id,,
0,0.745098,0.6
46,0.826059,0.6
76,0.697410,0.6
96,0.470387,0.6
224,0.786379,0.6


Let's join the target variable.

In [27]:
targets = pd.read_csv('data/train.csv', usecols=['row_id', 'answered_correctly'], index_col='row_id', nrows=nrows)
features = features.join(targets)
targets = features.pop('answered_correctly')
features.head()

/Users/mhalford/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,question_difficulty,avg_correct
row_id,,
0,0.745098,0.6
46,0.826059,0.6
76,0.697410,0.6
96,0.470387,0.6
224,0.786379,0.6


In [36]:
import numpy as np
from sklearn import model_selection

samples = np.random.choice(targets.index, size=500_000, replace=False)
X_fit, X_val, y_fit, y_val = model_selection.train_test_split(
    features.loc[samples], targets.loc[samples],
    random_state=42
)

#X_fit, X_val, y_fit, y_val = model_selection.train_test_split(
#    features, targets,
#    random_state=42
#)

In [37]:
import lightgbm as lgb

fit = lgb.Dataset(X_fit, y_fit)
val = lgb.Dataset(X_val, y_val, reference=fit)

model = lgb.train(
    params={
        'learning_rate': 0.01,
        'objective': 'binary',
        'metric': 'auc'
    },
    train_set=fit,
    num_boost_round=10_000,
    valid_sets=(fit, val),
    valid_names=('fit', 'val'),
    early_stopping_rounds=20,
    verbose_eval=100
)

Training until validation scores don't improve for 20 rounds
[100]	fit's auc: 0.747467	val's auc: 0.744844
[200]	fit's auc: 0.747876	val's auc: 0.745267
[300]	fit's auc: 0.748093	val's auc: 0.745436
[400]	fit's auc: 0.748326	val's auc: 0.745489
Early stopping, best iteration is:
[389]	fit's auc: 0.7483	val's auc: 0.74549


In [38]:
model.save_model('extractors/model.lgb')